<a href="https://colab.research.google.com/github/frank-895/train_learner/blob/main/learner_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

# Manual Linear Learner Model to Classify Handwritten Digits

## Extracting Data

MNIST contains images of handwritten digits. We will use these images to manually create a Learner model that can detect the difference between handwritten numbers.

In [3]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
(path).ls()

(#2) [Path('training'),Path('testing')]

We will download the training and validation images for each number into respective arrays, converting them to a scale of 0 to 1.

In [4]:
train_tns = []
valid_tns = []
# extract and convert to scale of 0 to 1 using tensor operations
for i in range(10):
  train_tns.append(torch.stack([tensor(Image.open(o)).float()/255 for o in (path/'training'/str(i)).ls()]))
  valid_tns.append(torch.stack([tensor(Image.open(o)).float()/255 for o in (path/'testing'/str(i)).ls()]))

Now, we need to convert our tensors from a list of matrices to a list of vectors, flattening the pixels in each image to a single row of 28*28 pixels. We use `view` which increases the axis to make it as big as needed to fit all the data. Then, we will create labels for each of the images using integer-encoded class labels for multi-class classification.

In [5]:
# extract images and flatten
train_x = torch.cat(train_tns).view(-1, 28*28)
valid_x = torch.cat(valid_tns).view(-1, 28*28)

# extract labels for each image
train_y = []
valid_y = []
for i in range(10):
  train_y.extend([i] * len(train_tns[i]))
  valid_y.extend([i] * len(valid_tns[i]))

# make labels rank-2 tensors
train_y = tensor(train_y).unsqueeze(1)
valid_y = tensor(valid_y).unsqueeze(1)
train_x.shape, train_y.shape

(torch.Size([60000, 784]), torch.Size([60000, 1]))

`.unsqueeze()` adds a new dimension to train_y, making it a rank-2 tensor, suitable for classification tasks. Because of this, it is easy to create a Dataset for PyTorch where each flattened image is associated with its label.

In [6]:
dset = list(zip(train_x, train_y))
valid_dset = list(zip(valid_x, valid_y))

Now we are ready to begin training our Learner model.


## Creating Model

### Initialise Weights

We start by initialising our **weights** with random values from the normal distribution. Each weight will be associated with one of the 28x28 pixel values. We give weights a shape in the 2nd axis of 10, representing the probability the image belongs to each of the 10 classes from 0 to 9. We also need to define bias to ensure the model is suitably flexible, otherwise input of 0 will not be trainable. Together, the weights and bias define our **parameters**.

In [18]:
def init_params(size, std=1.0):
  """Returns init"""
  return (torch.randn(size)*std).requires_grad_()

weights = init_params((28*28, 10))
bias = init_params(10)

params = weights, bias
weights.shape, bias.shape

(torch.Size([784, 10]), torch.Size([10]))

### Make a Prediction

Now that we have parameters and input, we can create a model that uses the parameters to classify each input.

We will use **matrix multiplication** for this task. We want to find the dot product of the weights and the 784 pixel values, then add the bias to this. We could do this in a number of ways, including using Python `for` loops - but this would be very inefficient. PyTorch is optimised to do the same task using matrix multiplication (`@` operator) extremely quickly on the GPU. We add the bias to the model too.

In [19]:
def matrix_x(x):
  """Uses the models parameters to predict the classification of an image."""
  return x@weights + bias

### Calculate Loss

Now, we calculate **loss** to determine how effective our parameters are in classifying handwritten digits. Loss is a value that represents how well (or badly) our model is doing.

Because we are performing multi-class classification we will use **cross-entropy loss**, which quantifies how well predicted probability distributions align with the true labels, penalizing incorrect predictions based on their confidence levels.

In [20]:
def calc_loss(preds, y):
  """Calculates the cross-entropy loss using a tensor of logits from the model"""
  return torch.nn.functional.cross_entropy(preds, y.squeeze())

### Calculate Gradient

Using **stochastic gradient descent** (SGD), which is an **optimiser** function, we will update the parameters in the model in each **epoch** to improve its predictive power. An epoch refers to a single complete pass of the entire training dataset through the model. SGD is an optimisation algorithm through which the parameters are updated by finding the gradient (derivative) of each parameter and gradually adjusting it so it moves towards its optimal value. PyTorch automatically employs **backpropagation** to calculate the gradient of the loss with respect to each weight using the chain rule of calculus because the linear_model is defined using `torch.nn`. PyTorch can very efficiently calculate derivatives.

We calculate the loss for **mini-batches** to ensure efficiency and suitable loss predictions. We will use a `DataLoader` object from PyTorch which turns a Python collection into an iterable, connecting each input with its respective label. It will also shuffle the data items in each mini-batch every epoch. The **batch size** is the number of data items in a mini-batch.

In [21]:
# make a function to compute the models predictions, calculate the loss and compute the gradients with respect to the loss
def calc_grad(xb, yb, model):
  preds = model(xb)
  loss = calc_loss(preds, yb)
  loss.backward()

### Step the Parameters

Now we can update each parameter (called "stepping") using the gradient calculated in `calc_grad` and the **learning rate** defined by `lr`.  The learning rate is size of step we take when applying SGD to update the parameters.

We will also create some code to define a metric for human consumption called accuracy. This will determine how accurately the model is correctly classifying the images and is more appropriate for human evaluation than loss (which is better for the training process).

In [22]:
# initialise parameters and 2 DataLoader objects for training set and validaiton set
dl = DataLoader(dset, batch_size=256)
valid_dl = DataLoader(valid_dset, batch_size=256)

def train_epoch(model, lr, params):
  """Update the parameters using the gradient of loss with respect to the parameter."""
  for x, y in dl:
    calc_grad(x, y, model)
    for p in params:
      p.data -= p.grad * lr
      p.grad.zero_() # reset gradient, as PyTorch accumulates by default

def batch_accuracy(x, y):
  preds = x.softmax(dim=1)
  predicted_classes = preds.argmax(dim=1)
  correct = predicted_classes == y
  return correct.float().mean()

def validate_epoch(model):
  accs = [batch_accuracy(model(xb), yb) for xb, yb in valid_dl]
  return round(torch.stack(accs).mean().item(), 4)

for i in range(20):
  train_epoch(matrix_x, 0.1, params)
  print(validate_epoch(matrix_x))

0.1763
0.2779
0.3708
0.436
0.4744
0.5022
0.5237
0.5437
0.5571
0.5701
0.5803
0.5882
0.5981
0.605
0.6114
0.6168
0.6223
0.628
0.6332
0.6374


This is now a working linear learner model working at about 63% accuracy. It is not yet a neural network as we have not introduced non-linearity - this is expected to drastically improve its predictive power.

### Activation Function

For the model to improve, we need to add more **layers** and non-linearity through **activation functions**. Linear classifiers are constrained in terms of their predictive power - to make it more complex to handle more tasks we need to make it a **neural network** (NN). A NN is a computational model inspired by the human brain, consisting of layers of interconnected nodes (neurons) that process and transform input data through weighted connections, enabling it to learn complex patterns and make predictions or classifications.

We will do this by making our model 3 layers.
1. Layer 1 will be a **linear layer**.
2. Layer 2 will be an **activation layer**.
3. Layer 3 will be a **linear layer**.

A **linear layer** also known as a **fully connected layer** linearly transforms the input by applying the parameters. This is what our linear model above does through the equation `y = WX + b` where:
- `y` is the transformed data.
- `W` is the weights matrix.
- `b` is the bias vector.
- `X` is the input features.

An **activation layer** introduces non-linearity in the model, enabling it to learn and model complex patterns. For example, the **Rectified Lienar Unit** (ReLU) function is a common activation function (makes negative values 0).

In [29]:
simple_net = nn.Sequential (
    nn.Linear(28*28, 30), # Layer 1
    nn.ReLU(), # Layer 2
    nn.Linear(30, 10) # Layer 3
)

In the first layer, we have 28*28 weights for each pixel in the MNIST images. We used 30 as a somewhat arbtirary hyperparameter (the number of neurons) as the output size. We would typically refine this parameter through experimentation. This value allows the model to learn more complex patterns.

In the second layer, we use the ReLU activation function to add nonlinearity.

In the third layer, we have 30 input, representing the 30 **activations** from  layer 1. Activations are the numbers that are calculated and returned by each linear or activation (non-linear) layer. The output size of 10 corresponds to the number of classes in the MNIST dataset (0 to 9). This layer will output a vector of 10 values, representing the network's "confidence" (**logits**) for each class.

We will be using the fastai model to make the NN. As such, we need to define a `DataLoaders` object with our training and validation sets.

In [33]:
dls = DataLoaders(dl, valid_dl)

In [35]:
learn = Learner(dls, simple_net, opt_func=SGD, loss_func=calc_loss, metrics=batch_accuracy)
learn.fit(20)

epoch,train_loss,valid_loss,batch_accuracy,time
0,1.741871,1.649581,0.609734,00:01
1,1.681991,1.576644,0.623380,00:01
2,1.621332,1.505009,0.636084,00:01
3,1.561149,1.435615,0.652066,00:00
4,1.501911,1.369272,0.664831,00:00
5,1.444192,1.306521,0.676434,00:00
6,1.388400,1.247706,0.687202,00:00
7,1.334945,1.192948,0.694479,00:00
8,1.284119,1.142238,0.701471,00:00
9,1.236120,1.095436,0.707382,00:00


By using a NN instead of a linear learner model, we have been able to improve the accuracy to about 75%. If we wanted to further improve our model, we could utilise pre-trained models such as resnet18. However, this process has demonstrated the predictive power of the NN.

While the model achieved reasonable accuracy, there is a risk of **overfitting**, especially as the model becomes more complex.Overfitting occurs when a model learns to perform exceptionally well on the training data but fails to generalize to new, unseen data, often capturing noise or irrelevant patterns instead of the underlying trends.

## Conclusion

This is why deep learning seems magical:
1. The neural network can solve any problem to any level of accuracy given the correct set of parameters.
2. There is a way to find the best set of parameters for any function called stochastic gradient descent.